In [ ]:
# 1.load data
# 2. text processing
# 3. vectorization(embedding)
# 4. ML algorithem to sentiment analysis

# output : product name, description, asin, reviews, sentiment analysis(percantile)


## 1.Load data

### Import Libraries

In [43]:
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import json
from pprint import pprint
from loguru import logger
from textblob import TextBlob

### Load API Key

In [16]:
load_dotenv()
API_KEY= os.getenv('rapidapi-key')

### product search and review urls

In [17]:
product_search_url = "https://real-time-amazon-data.p.rapidapi.com/search"
product_review_url = "https://real-time-amazon-data.p.rapidapi.com/product-reviews"

### Product Search Function

In [25]:
# create Product Search Function
def create_product_search(product_search_url, product):
    url = product_search_url
    querystring = {"query":product,"page":"1","country":"US","sort_by":"RELEVANCE","product_condition":"ALL"}

    headers = {
        "x-rapidapi-key": API_KEY,
        "x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    response_search = response.json() 
    logger.info(f'response status: {response.status_code}')
    asin_no = response_search['data']['products'][0]['asin']

    return response_search['data']['products']

In [26]:
phone = create_product_search(product_search_url, 'phone')

2024-07-31 15:04:30.845 | INFO     | __main__:create_product_search:13 - response status: 429


KeyError: 'data'

In [66]:
def get_product_info(product_search_url, product):
    response_search = create_product_search(product_search_url, product)
    product_info = []
    for i in range(len(response_search)):

        dict_ = {
            'No:': i,
            'asin': response_search[i]['asin'],
            'product_title': response_search[i]['product_title'],
            'num_ratings': response_search[i]['product_num_ratings']
        }
        product_info.append(dict_)
        
    product_info_df = pd.DataFrame(product_info)


    return product_info_df

In [67]:
product_infos_df = get_product_info(product_search_url, phone)

In [27]:
product_infos_df.head()

NameError: name 'product_infos_df' is not defined

In [72]:
product_infos_df.shape[0]

48

### Product Review Function

In [81]:
# Create Product Review Function
def create_product_review(product_review_url, product_search_url, product):
	product_info_df = get_product_info(product_search_url, product)
	for i in range(product_info_df.shape[0]):

		querystring = {"asin":product_info_df['asin'][i],"country":"US","sort_by":"TOP_REVIEWS","star_rating":"ALL","verified_purchases_only":"false","images_or_videos_only":"false","current_format_only":"false","page":"1"}

		headers = {
			"x-rapidapi-key": API_KEY,
			"x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
		}

		response_ = requests.get(product_review_url, headers=headers, params=querystring)
		response_search = response_.json()
		# reviews_responce = response_search['data']['reviews']

	return response_search

In [82]:
response_search = create_product_review(product_review_url, product_search_url, phone)

KeyError: 'data'

In [ ]:
reviews_responce

[]

In [ ]:
# Create Product Review Function
def create_product_review(product_review_url, product_search_url, product):
	asin_no = create_product_search(product_search_url, product)


	querystring = {"asin":asin_no,"country":"US","sort_by":"TOP_REVIEWS","star_rating":"ALL","verified_purchases_only":"false","images_or_videos_only":"false","current_format_only":"false","page":"1"}

	headers = {
		"x-rapidapi-key": API_KEY,
		"x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
	}

	response_ = requests.get(product_review_url, headers=headers, params=querystring)
	response_search = response_.json()
	reviews_responce = response_search['data']['reviews']

	return reviews_responce

In [54]:
def get_review_comments(product_review_url, product_search_url, product):
    reviews_responce = create_product_review(product_review_url, product_search_url, product)
  

    # review_lst = []
    # for item in reviews_responce[0]['review_comment']:
    #     review_lst.append(item)
        


    review_comment_lst = []
    for i in range(len(reviews_responce)):
        
        review_comment_dict = {
            'No': i+1,
            'review comment': reviews_responce[i]['review_comment']
        }
        review_comment_lst.append(review_comment_dict)

    return review_comment_lst

In [55]:
def get_review_comments(product_review_url, product_search_url, product):
    reviews_responce = create_product_review(product_review_url, product_search_url, product)
  

    # review_lst = []
    # for item in reviews_responce[0]['review_comment']:
    #     review_lst.append(item)
        


    review_comment_lst = []
    for i in range(len(reviews_responce)):
        
        review_comment_dict = {
            'No': i+1,
            'review comment': reviews_responce[i]['review_comment']
        }
        review_comment_lst.append(review_comment_dict)

    return review_comment_lst

In [56]:
reviews = get_review_comments(product_review_url, product_search_url, 'phone')

NameError: name 'create_product_review' is not defined

### Save Reviews

In [ ]:
def save_review(review_comments):
    
    with open('reviews.json', 'w') as f:
        json.dump(review_comments, f, indent=4)

In [ ]:
save_review(reviews)

## Load table

In [28]:
df = pd.read_csv('src/data/amazon_scraping_result.csv')

In [31]:
df.shape

(10, 21)

In [39]:
df[df['asin'] == 'B0CMT39J9B']

,Unnamed: 0,asin,product_title,product_price,product_original_price,currency,product_star_rating,product_num_ratings,product_url,product_photo,...,product_minimum_offer_price,is_best_seller,is_amazon_choice,is_prime,product_availability,climate_pledge_friendly,sales_volume,delivery,has_variations,reviews
1,1,B0CMT39J9B,"Apple iPhone 15 Plus, 128GB, Blue - Unlocked (...",$697.83,$750.00,USD,4.6,272,https://www.amazon.com/dp/B0CMT39J9B,https://m.media-amazon.com/images/I/51-RPOw693...,...,$669.99,False,False,False,Only 2 left in stock - order soon.,False,100+ bought in past month,"FREE delivery Sat, Aug 3 Or fastest delivery F...",True,"[""I was skeptical of a refurbished phone, but ..."


In [40]:
selected_df = df[['asin', 'product_title', 'product_num_ratings', 'reviews']]

In [47]:
selected_df

,asin,product_title,product_num_ratings,reviews
0,B0CMPLL2QT,"Apple iPhone 15, 128GB, Pink - T-Mobile (Renewed)",435,['Phone came really fast. I bought the phone o...
1,B0CMT39J9B,"Apple iPhone 15 Plus, 128GB, Blue - Unlocked (...",272,"[""I was skeptical of a refurbished phone, but ..."
2,B0CRJ1JNNS,"Apple iPhone 15, 128GB, Blue - Unlocked (Renew...",41,"['beautiful phone, no damage or scratches anyw..."
3,B0CMYXFK3R,"Apple iPhone 15 Pro, 256GB, Blue Titanium - Un...",309,"[""I'm coming from an S22 to the IPhone 15 Pro,..."
4,B0CLMGHDBV,Apple iPhone 15 Pro Max / 256GB / Black Titani...,17,"['Buen producto y nuevo', 'Happy with the prod..."
5,B0CRJJBBPY,"Apple iPhone 15 Pro Max, 256GB, Black Titanium...",8,"['It does get hot initially when charging, but..."
6,B0CRJNP7DH,"Apple iPhone 15, 128GB, Black - T-Mobile (Rene...",0,[]
7,B0CMZDBVWT,"Apple iPhone 15 Pro Max, 256GB, Blue Titanium ...",234,['It took time to convince myself to try a ref...
8,B0CRJTPR6T,"Apple iPhone 15 Pro Max, 256GB, White Titanium...",22,['i was needed a phone and thought I will Try ...
9,B0CRJ2XRBJ,"Apple iPhone 15 Plus, 256GB, Black - Unlocked ...",2,['I was a little scared because I’ve never pur...


In [54]:
sample_review = eval(selected_df[selected_df['asin'] == 'B0CMPLL2QT']['reviews'][0])

['Phone came really fast. I bought the phone on July 20th, phone was expected to arrive by Wednesday but instead came by today which is Monday. Phone is in excellent condition and everything works perfectly. It doesn’t come with the original box or charger however a charger is provided which charges really fast! I would definitely buy again! Also comes with no scratches whatsoever!',
 'I ordered this phone because it was cheaper than a renewed one on the Apple site. I did watch a lot of videos about fake iPhone 15. And this one passed all the tests. I was happy to see that the battery is 100% capacity. And it only had eight cycles when I got it. There was a tiny bit of scratching on the screen, but with the screen protector I put on I can’t even tell. I think I will keep getting Amazon renewed phones from now on. My last phone was an iPhone RX so this is a huge upgrade.',
 'I bought a iphone 15 for my son and it came on time the phone works great tmobile was surprised to see a phone ac

In [129]:
def get_review_comments(df, asin_no):

    reviews = []
    for i in range(df.shape[0]):   
        comment = eval(df[df['asin'] == asin_no]['reviews'][0])
        reviews.append(comment)
    
    return reviews

In [130]:
reviewes_df = get_review_comments(selected_df, 'B0CMPLL2QT')

In [132]:
len(reviewes_df)

10

In [133]:
reviewes_df


[['Phone came really fast. I bought the phone on July 20th, phone was expected to arrive by Wednesday but instead came by today which is Monday. Phone is in excellent condition and everything works perfectly. It doesn’t come with the original box or charger however a charger is provided which charges really fast! I would definitely buy again! Also comes with no scratches whatsoever!',
  'I ordered this phone because it was cheaper than a renewed one on the Apple site. I did watch a lot of videos about fake iPhone 15. And this one passed all the tests. I was happy to see that the battery is 100% capacity. And it only had eight cycles when I got it. There was a tiny bit of scratching on the screen, but with the screen protector I put on I can’t even tell. I think I will keep getting Amazon renewed phones from now on. My last phone was an iPhone RX so this is a huge upgrade.',
  'I bought a iphone 15 for my son and it came on time the phone works great tmobile was surprised to see a phone

In [80]:

# Sample DataFrame
sample_review = eval(selected_df[selected_df['asin'] == 'B0CMPLL2QT']['reviews'][0])
data = {'text': eval(selected_df[selected_df['asin'] == 'B0CMPLL2QT']['reviews'][0])}
df = pd.DataFrame(data)

# Function to get sentiment
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity


def get_sentiment_category(subjectivity):
        if subjectivity >= 0.5:
            return 'Positive'            
        else:
            return 'Negative'

# Apply the function to the DataFrame
df[['polarity', 'subjectivity']] = df['text'].apply(lambda x: pd.Series(get_sentiment(x)))
df['sentiment-category'] = df['subjectivity'].apply(lambda x: pd.Series(get_sentiment_category(x)))

print(df)

                                                text  polarity  subjectivity  \
0  Phone came really fast. I bought the phone on ...  0.340625      0.606250   
1  I ordered this phone because it was cheaper th...  0.116667      0.744444   
2  I bought a iphone 15 for my son and it came on...  0.367857      0.575000   
3  My renewed iphone came with very minimal defec...  0.045000      0.613333   
4  Upgraded from my iPhone 11 that was suffering ...  0.187846      0.510815   
5  It was time for me to upgrade my old iPhone 7 ...  0.225189      0.472680   
6  Screen shows some pink lines in the sunlight b...  0.158333      0.400000   
7  Phone came as described. Color was great, batt...  0.800000      0.750000   
8  I just bought the iPhone 15 with 256Gb memory ...  0.348295      0.588961   
9  Returned waiting for return from Affirm, who h...  0.522222      0.694444   

  sentiment-category  
0           Positive  
1           Positive  
2           Positive  
3           Positive  
4   